# S&P PyTechnicalIndicators study
This notebook is a use case of how to use the [PyTechnicalIndicators]() package.

The data used is S&P 500 prices from the 13th of October 2022 to the 12th of October 2023.

The notebook will go through every function in the PyTechnicalIndicators package. The package is broken down into three parts, Bulk, Single, and Chart Patterns. The Bulk and Single functions are identical with the exception that Bulk functions will expect to have lists sent to them and will return lists. The Single functions will return singular data. The aim is for the bulk functions to be used on datasets of multiple days, and the single functions to increment on those. This will be demonstrated below.
The Chart Pattern functions are used to highlight what has happened on charts, their purpose is to be added to graphs. This will also be demonstrated below.

To use this notebook for your own data replace `example.csv` with your own data. The data is expected to be tab (`\t`) separated rather than comma (`,`) separated. But that can be changed on line 9 in the cell below.

## Formatting the raw data
The cell below will just prepare the CSV for us, print and chart out the data so that we have an initial view of what the data holds.

`data['Typical Price']` is used for a number of calculations in the PyTechnicalIndicators package.

Lines 13 to 18 will be used to add singular data to the bulk analysis that will be done on `data`

In [23]:
import datetime
import math

import pandas
import plotly.graph_objects as go


data = pandas.read_csv("example.csv", sep='\t', index_col=0, parse_dates=True)
data.index.name = 'Date'
data['Typical Price'] = (data['High'] + data['Low'] + data['Close']) / 3
data.sort_index(inplace=True)

latest_date = datetime.date(2023, 10, 12)
latest_open = 4380.94
latest_high = 4382.67
latest_low = 4370.34
latest_close = 4370.56
latest_typical_price = (latest_high + latest_close + latest_low) / 3
latest_volume = 184806058

print(data)

# fig = go.Figure(data=go.Candlestick(
#     x=data.index,
#     open=data['Open'],
#     low=data['Low'],
#     high=data['High'],
#     close=data['Close'],
#     name='S&P 500'
# ))
# fig.add_scatter(
#     x=data.index,
#     y=data['Typical Price'],
#     name='Typical Price',
#     line={'color': 'blue'}
# )
# fig.show()

               Open     High      Low    Close      Volume  Typical Price
Date                                                                     
2022-10-13  3520.37  3685.41  3491.58  3669.91  5021680000    3615.633333
2022-10-14  3690.41  3712.00  3579.68  3583.07  4243030000    3624.916667
2022-10-17  3638.65  3689.73  3638.65  3677.95  4352780000    3668.776667
2022-10-18  3746.26  3762.79  3686.53  3719.98  4483740000    3723.100000
2022-10-19  3703.11  3728.58  3666.51  3695.16  4223800000    3696.750000
...             ...      ...      ...      ...         ...            ...
2023-10-05  4259.31  4267.13  4225.91  4258.19  3581470000    4250.410000
2023-10-06  4234.79  4324.10  4219.55  4308.50  3902030000    4284.050000
2023-10-09  4289.02  4341.73  4283.79  4335.66  3174630000    4320.393333
2023-10-10  4339.75  4385.46  4339.64  4358.24  3520240000    4361.113333
2023-10-11  4366.59  4378.64  4345.34  4376.95  3601660000    4366.976667

[250 rows x 6 columns]


## The basic indicators
In this section we will cover the basic indicators.

A number of these don't have a single version as they are just calling functions in the `math` and `statisitics` packages.

In [24]:
from PyTechnicalIndicators.Bulk import basic_indicators as bulk_basic_indicators
from PyTechnicalIndicators.Single import basic_indicators as single_basic_indicator

### Mean
The mean (aka the average) simply tells you what the mean price the asset has been trading at for a given period. 

The period is determined by the function caller, it is the number of days the caller wants an average for. Common favorites are weekly, monthly, quarterly averages. Keeping in mind that we are looking at the S&P, which isn't traded on weekends, so a week is 5 days. If looking at cryptocurrencies the function call would need to have a period of 7. The below call also doesn't take into account bank holidays, it could be refined further but serves its purpose as an example.

The price used for the mean could be either Open, High, Low, Close, or Typical Price. The most commonly used is the typical price.

In [25]:
weekly_mean = bulk_basic_indicators.mean(data['Typical Price'], 5)
monthly_mean = bulk_basic_indicators.mean(data['Typical Price'], 20)
quarterly_mean = bulk_basic_indicators.mean(data['Typical Price'], 60)

print(weekly_mean)

import statistics
weekly_mean.append(statistics.mean(data['Typical Price'][-4:]+[latest_typical_price]))
monthly_mean.append(statistics.mean(data['Typical Price'][-19:]+[latest_typical_price]))
quarterly_mean.append(statistics.mean(data['Typical Price'][-59:]+[latest_typical_price]))

# TODO create a new DF and append the means at the end

[3665.8353333333334, 3679.923333333333, 3698.8106666666667, 3721.704, 3745.1773333333335, 3775.2153333333335, 3802.7366666666667, 3833.1820000000002, 3851.7926666666667, 3857.812, 3849.2780000000002, 3829.117333333333, 3806.5166666666664, 3790.287333333333, 3781.094, 3774.306, 3814.7033333333334, 3858.9366666666665, 3894.748, 3928.078, 3967.0933333333332, 3969.2826666666665, 3965.4146666666666, 3960.2586666666666, 3959.7653333333333, 3970.664, 3988.8953333333334, 3992.28, 3994.0646666666667, 4002.8953333333334, 4014.112, 4020.6393333333335, 4027.5593333333336, 4026.8333333333335, 4007.858, 3984.23, 3961.9386666666664, 3953.9913333333334, 3970.8233333333333, 3984.194, 3974.8633333333332, 3956.6459999999997, 3927.045333333333, 3883.168, 3856.036, 3836.4739999999997, 3830.904, 3832.008, 3829.1306666666665, 3822.808666666667, 3825.4453333333336, 3826.0653333333335, 3829.584, 3832.1133333333332, 3838.6406666666667, 3855.5333333333333, 3870.1626666666666, 3891.9173333333333, 3923.13266666666

/tmp/ipykernel_4091/3105153410.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  weekly_mean.append(statistics.mean(data['Typical Price'][-4]+[latest_typical_price]))
/tmp/ipykernel_4091/3105153410.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  monthly_mean.append(statistics.mean(data['Typical Price'][-19]+[latest_typical_price]))
/tmp/ipykernel_4091/3105153410.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quarterly_me

### Median
The median is the middle value of series of prices. 

The median is an alternative to the mean, it is believed to be more significant than the mean, as the mean includes prices that could be seen as extremes. Whereas the median returns the middle value where most the prices are most likely to be around.

Similarily to the mean, the median needs a list of prices and a period. We will once again use the Typical Price and get the weekly, monthly, and quarterly medians


In [26]:
weekly_median = bulk_basic_indicators.median(data['Typical Price'], 5)
monthly_median = bulk_basic_indicators.median(data['Typical Price'], 20)
quarterly_median = bulk_basic_indicators.median(data['Typical Price'], 60)

print(weekly_median)

weekly_median.append(statistics.median(data['Typical Price'][-4:]+[latest_typical_price]))
monthly_median.append(statistics.median(data['Typical Price'][-19:]+[latest_typical_price]))
quarterly_median.append(statistics.median(data['Typical Price'][-59:]+[latest_typical_price]))

[3668.7766666666666, 3686.0733333333337, 3696.75, 3719.353333333333, 3719.353333333333, 3783.2433333333333, 3823.6800000000003, 3840.4666666666667, 3846.94, 3870.5633333333335, 3870.5633333333335, 3870.5633333333335, 3804.27, 3795.15, 3795.15, 3770.33, 3795.15, 3824.596666666667, 3924.8633333333332, 3974.2066666666665, 3974.2066666666665, 3974.2066666666665, 3965.406666666667, 3960.403333333333, 3960.403333333333, 3960.403333333333, 3988.78, 3988.78, 3988.78, 4019.9, 4026.966666666667, 4032.933333333334, 4032.933333333334, 4032.933333333334, 4011.9266666666663, 3957.8433333333337, 3953.72, 3953.72, 3957.8433333333337, 3972.19, 3972.19, 3972.19, 3911.19, 3857.06, 3857.06, 3824.1866666666665, 3824.1866666666665, 3829.2100000000005, 3829.2100000000005, 3829.2100000000005, 3829.2100000000005, 3829.7066666666665, 3832.31, 3832.31, 3832.31, 3847.2999999999997, 3870.2766666666666, 3905.4566666666665, 3911.0266666666666, 3956.0733333333337, 3972.83, 3972.83, 3972.83, 3956.536666666667, 3956.53

/tmp/ipykernel_4091/2297098699.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  weekly_median.append(statistics.median(data['Typical Price'][-4]+[latest_typical_price]))
/tmp/ipykernel_4091/2297098699.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  monthly_median.append(statistics.median(data['Typical Price'][-19]+[latest_typical_price]))
/tmp/ipykernel_4091/2297098699.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quart

### Standard Deviation
The standard deviation shows how far from the average.

This helps determine how volatile the market is. The further prices tend to be from the average the more volatile the market is.

In [27]:
weekly_standard_deviation = bulk_basic_indicators.standard_deviation(data['Typical Price'], 5)
monthly_standard_deviation = bulk_basic_indicators.standard_deviation(data['Typical Price'], 20)
quarterly_standard_deviation = bulk_basic_indicators.standard_deviation(data['Typical Price'], 60)

print(weekly_standard_deviation)

weekly_median.append(statistics.stdev(data['Typical Price'][-4:]+[latest_typical_price]))
monthly_median.append(statistics.stdev(data['Typical Price'][-19:]+[latest_typical_price]))
quarterly_median.append(statistics.stdev(data['Typical Price'][-59:]+[latest_typical_price]))

[45.9298056579579, 36.52110797576905, 22.806249314119395, 37.705472149278194, 65.25803745303881, 71.64752665344109, 52.79413109217535, 32.790676147682895, 21.98939092683851, 22.2319085750388, 33.02056418186833, 66.42892385433059, 67.58358643602432, 55.25658565074206, 40.383995344690824, 38.3139019156234, 66.5857190227588, 89.49817304156421, 93.3352398495754, 91.76356383542311, 25.39794906417967, 20.925650580620406, 20.286333467523228, 20.76362505815287, 19.852618998789954, 33.78962393332546, 34.85769687043483, 32.1130669146794, 29.180734854809252, 33.537305066315795, 47.22984311969642, 51.511699825271926, 46.29879140491227, 47.68273697634798, 61.51898851393609, 50.515684967916535, 28.912313851060322, 12.591432845832625, 39.5437427976214, 36.909359050998845, 49.114974351571725, 72.72140627078099, 92.29907019514809, 77.37826713978257, 37.55480642415242, 25.02831068121778, 22.245916154556518, 21.964403601180667, 24.920789781492424, 13.667410874046427, 12.625670191232569, 12.92922198398306

### Variance
The variance shows how spread out the prices from one another.

This helps determine how volatile the market is. The more spread out they are, the more volatile the market.

In [28]:
weekly_variance = bulk_basic_indicators.variance(data['Typical Price'], 5)
monthly_variance = bulk_basic_indicators.variance(data['Typical Price'], 20)
quarterly_variance = bulk_basic_indicators.variance(data['Typical Price'], 60)

print(weekly_variance)

weekly_variance.append(statistics.variance(data['Typical Price'][-4:]+[latest_typical_price]))
monthly_variance.append(statistics.variance(data['Typical Price'][-19:]+[latest_typical_price]))
quarterly_variance.append(statistics.variance(data['Typical Price'][-59:]+[latest_typical_price]))

[2109.5470477777812, 1333.7913277777814, 520.1250077777713, 1421.7026299999936, 4258.611452222216, 5133.368075555551, 2787.220277777796, 1075.2284422222199, 483.5333133333277, 494.2577588888838, 1090.357658888886, 4412.801924444452, 4567.541155555571, 3053.290257777793, 1630.8670800000102, 1467.95508000001, 4433.6579777777815, 8009.922977777773, 8711.466997777768, 8420.551647777773, 645.055816666665, 437.8828522222192, 411.53532555555296, 431.1281255555537, 394.12648111111577, 1141.7386855555606, 1215.0590311111223, 1031.2490666666768, 851.5152866666795, 1124.7508311111312, 2230.658081111135, 2653.4552188889197, 2143.5780855555786, 2273.643405555583, 3784.5859477778, 2551.8344277777887, 835.9218922222145, 158.54418111111266, 1563.7075944444343, 1362.3007855555502, 2412.2807055555486, 5288.402929999985, 8519.118358888874, 5987.396225555553, 1410.3634855555597, 626.41633555556, 494.8807855555587, 482.43502555555824, 621.0457633333373, 186.7981200000025, 159.40754777777866, 167.1647811111

### Mean absolute deviation
The mean absolute deviation also measures how much the prices deviate from the mean but uses the absolute value of the difference as opposed to the difference.

This also helps to determine market volatility.

In [29]:
weekly_mean_absolute_deviation = bulk_basic_indicators.mean_absolute_deviation(data['Typical Price'], 5)
monthly_mean_absolute_deviation = bulk_basic_indicators.mean_absolute_deviation(data['Typical Price'], 20)
quarterly_mean_absolute_deviation = bulk_basic_indicators.mean_absolute_deviation(data['Typical Price'], 60)

print(weekly_mean_absolute_deviation)

weekly_mean_absolute_deviation.append(single_basic_indicator.mean_absolute_deviation(data['Typical Price'][-4:]+[latest_typical_price]))
monthly_mean_absolute_deviation.append(single_basic_indicator.mean_absolute_deviation(data['Typical Price'][-19:]+[latest_typical_price]))
quarterly_mean_absolute_deviation.append(single_basic_indicator.mean_absolute_deviation(data['Typical Price'][-59:]+[latest_typical_price]))

[36.448266666666676, 26.46133333333346, 17.93279999999986, 25.174133333333337, 53.342133333333365, 58.00159999999996, 41.1506666666668, 23.77626666666656, 17.716533333333246, 18.00159999999996, 28.242399999999908, 52.43520000000008, 53.53066666666673, 39.64853333333349, 32.29386666666678, 28.45386666666682, 48.0213333333334, 74.69333333333324, 77.82773333333326, 64.38506666666663, 17.566666666666514, 14.939466666666522, 13.849599999999828, 14.51226666666671, 14.11760000000013, 26.940800000000127, 27.6304000000001, 24.922666666666828, 23.494933333333485, 28.445600000000287, 37.41893333333364, 42.64080000000031, 34.33680000000031, 35.208000000000354, 49.57573333333339, 41.22799999999997, 19.99519999999984, 8.820266666666885, 27.36933333333327, 29.760799999999925, 37.22533333333331, 58.01679999999997, 75.4797333333332, 59.90560000000005, 27.75680000000002, 21.34480000000003, 15.338400000000002, 14.896799999999894, 16.309866666666856, 11.252266666666765, 8.535466666666617, 8.78346666666657

### Median absolute deviation
Similar to the mean absolute deviation but instead of comparing how much the prices deviate from the mean, it compares how much the prices deviate from the median.

This also helps to determine market volatility.

In [30]:
weekly_median_absolute_deviation = bulk_basic_indicators.median_absolute_deviation(data['Typical Price'], 5)
monthly_median_absolute_deviation = bulk_basic_indicators.median_absolute_deviation(data['Typical Price'], 20)
quarterly_median_absolute_deviation = bulk_basic_indicators.median_absolute_deviation(data['Typical Price'], 60)

print(weekly_median_absolute_deviation)

weekly_median_absolute_deviation.append(single_basic_indicator.median_absolute_deviation(data['Typical Price'][-4:]+[latest_typical_price]))
monthly_median_absolute_deviation.append(single_basic_indicator.median_absolute_deviation(data['Typical Price'][-19:]+[latest_typical_price]))
quarterly_median_absolute_deviation.append(single_basic_indicator.median_absolute_deviation(data['Typical Price'][-59:]+[latest_typical_price]))

[35.860000000000035, 25.23133333333335, 17.52066666666651, 24.703999999999905, 48.177333333333266, 56.396, 36.96200000000008, 22.31933333333327, 16.745999999999913, 15.451333333333242, 23.985333333333255, 44.146, 53.08133333333344, 38.676000000000116, 29.482666666666773, 27.658666666666797, 44.110666666666745, 67.8253333333333, 71.80466666666662, 55.15933333333332, 16.14399999999987, 13.954666666666526, 13.847999999999956, 14.483333333333395, 13.990000000000146, 24.888666666666722, 27.607333333333465, 24.22266666666683, 22.438000000000194, 25.044666666666945, 34.848000000000226, 40.182000000000244, 33.26200000000026, 33.988000000000284, 48.76200000000017, 35.95066666666671, 16.713333333333413, 8.766000000000167, 24.77333333333336, 27.359999999999946, 36.69066666666667, 54.9079999999999, 72.30866666666661, 54.68400000000001, 27.552000000000042, 18.88733333333339, 13.3173333333335, 13.416666666666696, 16.29400000000005, 9.972000000000026, 7.3353333333332555, 7.855999999999949, 10.8539999

### Mode absolute deviation
Similar to the mean absolute deviation but instead of comparing how much the prices deviate from the mean, it compares how much the prices deviate from the mode (most occurring value).

This also helps to determine market volatility.

In [31]:
weekly_mode_absolute_deviation = bulk_basic_indicators.mode_absolute_deviation(data['Typical Price'], 5)
monthly_mode_absolute_deviation = bulk_basic_indicators.mode_absolute_deviation(data['Typical Price'], 20)
quarterly_mode_absolute_deviation = bulk_basic_indicators.mode_absolute_deviation(data['Typical Price'], 60)

print(weekly_mode_absolute_deviation)

weekly_mode_absolute_deviation.append(single_basic_indicator.mode_absolute_deviation(data['Typical Price'][-4:]+[latest_typical_price]))
monthly_mode_absolute_deviation.append(single_basic_indicator.mode_absolute_deviation(data['Typical Price'][-19:]+[latest_typical_price]))
quarterly_mode_absolute_deviation.append(single_basic_indicator.mode_absolute_deviation(data['Typical Price'][-59:]+[latest_typical_price]))

[50.20200000000004, 55.006666666666845, 30.034000000000013, 25.453333333333283, 52.697999999999865, 89.14199999999964, 83.38333333333367, 49.93866666666672, 18.040666666666585, 20.17599999999993, 33.361999999999895, 44.34933333333329, 69.78000000000002, 80.27600000000021, 31.30666666666675, 51.429333333333396, 56.126666666666914, 73.71466666666666, 91.8579999999999, 157.74799999999996, 42.23000000000011, 15.061999999999898, 15.60799999999981, 30.987999999999737, 14.990666666666948, 34.854000000000084, 33.28266666666686, 43.853333333333374, 22.438000000000194, 25.044666666666945, 34.848000000000226, 51.30333333333374, 70.2093333333336, 33.988000000000284, 68.12533333333349, 75.37333333333382, 49.98799999999974, 8.766000000000167, 32.766666666666424, 30.229333333333216, 41.49933333333347, 54.9079999999999, 110.83466666666645, 121.74199999999992, 55.15400000000009, 25.46200000000008, 13.3173333333335, 15.560000000000127, 40.11933333333336, 13.138000000000194, 7.3353333333332555, 7.8559999

### Logarithm
The log of the price 

TODO: Figure why it is used

In [32]:
log = bulk_basic_indicators.log(data['Typical Price'])

print(log)

log.append(math.log(latest_typical_price))

[8.193022315190678, 8.195586578518789, 8.207613552147379, 8.222311933571085, 8.215209334253597, 8.212317033130436, 8.221305097030667, 8.238336945412232, 8.253349165975647, 8.255033306061913, 8.24896858866139, 8.261417971438675, 8.262635509469352, 8.261155339599211, 8.243879399064449, 8.222251945917778, 8.231795618617989, 8.24147921474031, 8.249208294077656, 8.234917809767753, 8.275086810077386, 8.288972607086743, 8.287580426456763, 8.291858908868397, 8.285363692912174, 8.277871984649273, 8.28410115091383, 8.28107246631149, 8.291240698717413, 8.299012305681773, 8.30076868335672, 8.28836518080019, 8.28332988832048, 8.302249264444981, 8.312867305358136, 8.308840546673284, 8.297026870416932, 8.282412186857908, 8.278442648802667, 8.283454543098498, 8.28100154947496, 8.28707285886492, 8.303475080749182, 8.29527638733985, 8.271596954501094, 8.25766051421894, 8.249101087484783, 8.24761120844066, 8.260815968792869, 8.246271213953037, 8.25041379458033, 8.25054349090397, 8.243836463931105, 8.2526

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/basic_indicators.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  logs.append(math.log(prices[i]))


### Log difference
The log difference measure the difference in price from one date to another.

It is another way to determine how volatile an asset is

In [33]:
log_diff = bulk_basic_indicators.log_diff(data['Typical Price'])

print(log_diff)

log_diff.append(log(data['Typical Price'][-1]) - log(latest_typical_price))

[0, 0.002564263328110883, 0.012026973628589843, 0.014698381423706053, -0.007102599317487801, -0.0028923011231611895, 0.008988063900231325, 0.01703184838156524, 0.015012220563415113, 0.0016841400862652733, -0.0060647174005232785, 0.012449382777285578, 0.0012175380306764794, -0.0014801698701401023, -0.017275940534762668, -0.021627453146670916, 0.009543672700210948, 0.009683596122322058, 0.007729079337345013, -0.014290484309903206, 0.04016900030963377, 0.013885797009356438, -0.0013921806299794781, 0.004278482411633533, -0.006495215956222822, -0.007491708262900687, 0.006229166264557051, -0.0030286846023397374, 0.01016823240592224, 0.007771606964359634, 0.001756377674947629, -0.012403502556530555, -0.005035292479709241, 0.018919376124500786, 0.010618040913154658, -0.0040267586848514725, -0.011813676256352323, -0.0146146835590244, -0.003969538055240918, 0.005011894295831709, -0.0024529936235389016, 0.006071309389961144, 0.016402221884261436, -0.00819869340933188, -0.023679432838756398, -0.01

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/basic_indicators.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  logdiffs.append(math.log(prices[i]) - math.log(prices[i - 1]))
/tmp/ipykernel_4091/456923825.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  log_diff.append(math.log(data['Typical Price'][-1]) - math.log(latest_typical_price))


## Moving Averages
One of the most common indicators when looking at stock prices

In [34]:
from PyTechnicalIndicators.Bulk import moving_averages as bulk_moving_average
from PyTechnicalIndicators.Single import moving_averages as single_moving_average

### Moving Average
The moving average provides a general overview of the direction of the price without the daily fluctuations by taking the average of a given period.

The period is determined by the caller.

Similar as above, we will be doing a weekly, monthly, and quarterly moving average.

In [35]:
weekly_ma = bulk_moving_average.moving_average(data['Typical Price'], 5)
monthly_ma = bulk_moving_average.moving_average(data['Typical Price'], 20)
quarterly_ma = bulk_moving_average.moving_average(data['Typical Price'], 60)

print(weekly_ma)

weekly_ma.append(single_moving_average.moving_average(data['Typical Price'][-4:] + [latest_typical_price]))
monthly_ma.append(single_moving_average.moving_average(data['Typical Price'][-19:] + [latest_typical_price]))
quarterly_ma.append(single_moving_average.moving_average(data['Typical Price'][-59:] + [latest_typical_price]))

[3665.8353333333334, 3679.9233333333336, 3698.8106666666667, 3721.704, 3745.177333333333, 3775.2153333333335, 3802.7366666666667, 3833.182, 3851.7926666666667, 3857.8120000000004, 3849.278, 3829.117333333333, 3806.5166666666673, 3790.2873333333337, 3781.094, 3774.3059999999996, 3814.7033333333334, 3858.936666666667, 3894.7479999999996, 3928.078, 3967.0933333333332, 3969.282666666667, 3965.4146666666666, 3960.258666666667, 3959.7653333333337, 3970.6639999999998, 3988.895333333334, 3992.28, 3994.0646666666667, 4002.895333333333, 4014.112, 4020.6393333333335, 4027.5593333333336, 4026.8333333333335, 4007.858, 3984.2300000000005, 3961.9386666666664, 3953.991333333333, 3970.8233333333337, 3984.1940000000004, 3974.8633333333332, 3956.6459999999997, 3927.045333333333, 3883.168, 3856.036, 3836.4739999999997, 3830.904, 3832.0080000000003, 3829.130666666667, 3822.808666666667, 3825.445333333334, 3826.0653333333335, 3829.584, 3832.1133333333332, 3838.640666666666, 3855.533333333333, 3870.162666666

### Smoothed moving average
The smoothed moving average is similar to the moving average but puts more weight onto the more recent values when calculating the average

In [ ]:
weekly_sma = bulk_moving_average.smoothed_moving_average(data['Typical Price'], 5)
monthly_sma = bulk_moving_average.smoothed_moving_average(data['Typical Price'], 20)
quarterly_sma = bulk_moving_average.smoothed_moving_average(data['Typical Price'], 60)

print(weekly_sma)

weekly_sma.append(single_moving_average.smoothed_moving_average(data['Typical Price'][-4:] + [latest_typical_price]))
monthly_sma.append(single_moving_average.smoothed_moving_average(data['Typical Price'][-19:] + [latest_typical_price]))
quarterly_sma.append(single_moving_average.smoothed_moving_average(data['Typical Price'][-59:] + [latest_typical_price]))

### Exponential moving average
The exponential moving average is similar to the moving average but puts more weight onto the more recent values when calculating the average

In [ ]:
weekly_ema = bulk_moving_average.exponential_moving_average(data['Typical Price'], 5)
monthly_ema = bulk_moving_average.exponential_moving_average(data['Typical Price'], 20)
quarterly_ema = bulk_moving_average.exponential_moving_average(data['Typical Price'], 60)

print(weekly_ema)

weekly_ema.append(single_moving_average.exponential_moving_average(data['Typical Price'][-4:] + [latest_typical_price]))
monthly_ema.append(single_moving_average.exponential_moving_average(data['Typical Price'][-19:] + [latest_typical_price]))
quarterly_ema.append(single_moving_average.exponential_moving_average(data['Typical Price'][-59:] + [latest_typical_price]))

### Personalised moving average
The personalised moving average allows the caller to put weight on the prices they want. 

It essentially exposes the inner workings of the smoothed and exponential moving averages. The key parameters for the personalised moving average are the `alpha_nominator` and `alpha_denominator`. The alpha determines the weight that will be applied to prices. 

For the smoothed moving average the nominator is 1 and denominator is 0. For the exponential moving average the alpha nominator is 2 and the denominator is 1. They both call this function with hardcoded values.

In [ ]:
weekly_pma = bulk_moving_average.personalised_moving_average(data['Typical Price'], 5, 4, 2)
monthly_pma = bulk_moving_average.personalised_moving_average(data['Typical Price'], 20, 4, 2)
quarterly_pma = bulk_moving_average.personalised_moving_average(data['Typical Price'], 60, 4, 2)

print(weekly_ema)

weekly_pma.append(single_moving_average.personalised_moving_average(data['Typical Price'][-4:] + [latest_typical_price], 4, 2))
monthly_pma.append(single_moving_average.personalised_moving_average(data['Typical Price'][-19:] + [latest_typical_price], 4, 2))
quarterly_pma.append(single_moving_average.personalised_moving_average(data['Typical Price'][-59:] + [latest_typical_price], 4, 2))

### Moving Average Convergence Divergence
The Moving Average Convergence Divergence (MACD) is an oscillator that calculates the difference between the 12 and 26 period exponential moving averages. It is plotted against the Signal line a 9 period exponential moving average of the MACD.

It produces "buy" and "sell" signals when the two lines cross one another.

In [ ]:
macd = bulk_moving_average.moving_average_convergence_divergence(data['Typical Price'])
signal = bulk_moving_average.signal_line(macd)

print(macd)
print(signal)

macd.append(single_moving_average.moving_average_convergence_divergence(data['Typical Price'][-25:] + [latest_typical_price]))
signal.append(single_moving_average.signal_line(macd[-9:]))

### Personalised MACD
The MACD makes a lot of assumptions that are no longer true. When it was invented the working week was 6 days, showing us how the two week moving average differed from the month moving average. As the work week is now 5 days (or 7 if trading cryptocurrencies) the personalised MACD allows the caller to personalise the MACD.

When calling `personalised_macd` the caller needs to provide the short period, the long period, and the moving average model they'd like to use.
When calling `personalised_signal` the caller needs to provide the period and the moving average model.

In [ ]:
personalised_macd = bulk_moving_average.personalised_macd(data['Typical Price'], 10, 20, 'sma')
personalised_signal = bulk_moving_average.personalised_signal_line(data['Typical Price'], 15, 'sma')

print(personalised_macd)
print(personalised_signal)

personalised_macd.append(single_moving_average.personalised_macd(data['Typical Price'][-20:] + [latest_typical_price], 10, 20, 'sma'))
personalised_signal.append(single_moving_average.personalised_signal_line(personalised_macd[-15:], 'sma'))

### McGinley Dynamic
The McGinley Dynamic is an alternative to the moving average, it was built to be more resilient to shocks.

It uses previously calculated McGiley dynamics to determine the new one, so when calling the single function, if one is available it should be provided to get an accurate value

In [ ]:
weekly_mcginley_dynamic = bulk_moving_average.mcginley_dynamic(data['Typical Price'], 5)
monthly_mcginley_dynamic = bulk_moving_average.mcginley_dynamic(data['Typical Price'], 20)
quarterly_mcginley_dynamic = bulk_moving_average.mcginley_dynamic(data['Typical Price'], 60)

print(weekly_mcginley_dynamic)

weekly_mcginley_dynamic.append(single_moving_average.mcginley_dynamic(latest_typical_price, 5, weekly_mcginley_dynamic[-1]))
monthly_mcginley_dynamic.append(single_moving_average.mcginley_dynamic(latest_typical_price, 20, monthly_mcginley_dynamic[-1]))
quarterly_mcginley_dynamic.append(single_moving_average.mcginley_dynamic(latest_typical_price, 60, quarterly_mcginley_dynamic[-1]))

### Moving Average Envelopes
The moving average envelope function provides a moving average bracket calculated by taking a provided percentage and applying it to the calculated moving average.

`moving_average_envelope` needs to be given a list of prices, the period to calculate the moving average, as well as optionally the moving average model (defaults to moving average), and the difference (defaults to 3%). 

In [ ]:
moving_average_envelope = bulk_moving_average.moving_average_envelopes(data['Typical Price'], 5, 'ema', 5)

print(moving_average_envelope)

moving_average_envelope.append(single_moving_average.moving_average_envelopes(data['Typical Price'][-4:] + [latest_typical_price], 'ema', 5))

## Oscillators
An oscillator in a technical indicator that oscillates between two predetermined limits.

In [ ]:
from PyTechnicalIndicators.Bulk import oscillators as bulk_oscillators
from PyTechnicalIndicators.Single import oscillators as single_oscillators


### Stochastic Oscillator
The stochastic oscillator compares closing prices for the last 14 periods. It takes the difference between the latest close, lowest close, and highest close.
The asset is generally considered overbought over 80 and oversold under 20, but varies from asset to asset. 

In [ ]:
stochastic_oscillator = bulk_oscillators.stochastic_oscillator(data['Close'])

print(stochastic_oscillator)

stochastic_oscillator.append(single_oscillators.stochastic_oscillator(data['Close'][-13:]+[latest_close]))

### Personalised Stochastic Oscillator
The stochastic oscillator being fixed to 14 periods is rather inflexible as different assets trade on different periods, the personalised stochastic oscillator lets the function caller to determine their period
The single version does not need a period as it just assumes that the length of the submitted list is the period.

In [ ]:
weekly_so = bulk_oscillators.personalised_stochastic_oscillator(data['Close'], 5)
monthly_so = bulk_oscillators.personalised_stochastic_oscillator(data['Close'], 20)
quarterly_so = bulk_oscillators.personalised_stochastic_oscillator(data['Close'], 60)

print(weekly_so)

weekly_so.append(single_oscillators.personalised_stochastic_oscillator(data['Close'][-4:]+[latest_close]))
monthly_so.append(single_oscillators.personalised_stochastic_oscillator(data['Close'][-19:]+[latest_close]))
quarterly_so.append(single_oscillators.personalised_stochastic_oscillator(data['Close'][-59:]+[latest_close]))

### Fast Stochastic
The fast stochastic is simply the moving average of the stochastic oscillator. 

The caller decides which moving average model they'd like to use and the period.

In [ ]:
# Dropping the last value to demonstrate the use of the single function. This would obviously not be done normally
fast_stochastic = bulk_oscillators.fast_stochastic(weekly_so[:-1], 5, 'ema')

print(fast_stochastic)

fast_stochastic.append(single_oscillators.fast_stochastic(weekly_so[-5:], 'ema'))

### Slow Stochastic
The slow stochastic is the moving average of the fast stochastic

The caller determines the moving average model and the period.

In [ ]:
# Once again the last item is dropped to demonstrate how the single version is called
slow_stochastic = bulk_oscillators.slow_stochastic(fast_stochastic[:-1], 5, 'ema')

print(slow_stochastic)

slow_stochastic.append(single_oscillators.slow_stochastic(fast_stochastic[-5:], 'ema'))

### Slow Stochastic DS
The slow stochastic ds is the moving average of the slow stochastic

The caller determines the moving average model and the period

In [ ]:
# Once again the last item is dropped to demonstrate how the single version is called
slow_stochastic_ds = bulk_oscillators.slow_stochastic_ds(slow_stochastic[:-1], 5, 'ema')

print(slow_stochastic_ds)

slow_stochastic_ds.append(single_oscillators.slow_stochastic_ds(slow_stochastic[-5:], 'ema'))

### Visualizing the stochastics
The stochastics should be charted onto the same chart to get the full picture of the movement of the closing price

In [ ]:
# TODO: Chart the stochastics together

### Money Flow Index
The money flow index is an oscillator that measures the relation between the price and the volume. It looks at the price and volume over the past 14 periods

The asset is generally considered overbought over 80 and oversold under 20, but this can vary from asset to asset

In [ ]:
money_flow_index = bulk_oscillators.money_flow_index(data['Typical Price'], data['Volume'])

print(money_flow_index)

money_flow_index.append(single_oscillators.money_flow_index(data['Typical Price'][-13:]+[latest_typical_price], data['Volume'][-13:]+[latest_volume]))

### Personalised Money Flow Index
The personalised money flow index is identical to the money flow index but allows the caller to determine the period for which the MFI is calculated.

In [ ]:
weekly_personalised_mfi = bulk_oscillators.personalised_money_flow_index(data['Typical Price'], data['Volume'], 5)
monthly_personalised_mfi = bulk_oscillators.personalised_money_flow_index(data['Typical Price'], data['Volume'], 20)
quarterly_personalised_mfi = bulk_oscillators.personalised_money_flow_index(data['Typical Price'], data['Volume'], 60)

print(weekly_personalised_mfi)

weekly_personalised_mfi.append(single_oscillators.personalised_money_flow_index(data['Typical Price'][-4:]+[latest_typical_price], data['Volume'][-4:]+[latest_volume]))
monthly_personalised_mfi.append(single_oscillators.personalised_money_flow_index(data['Typical Price'][-19:]+[latest_typical_price], data['Volume'][-19:]+[latest_volume]))
quarterly_personalised_mfi.append(single_oscillators.personalised_money_flow_index(data['Typical Price'][-59:]+[latest_typical_price], data['Volume'][-59:]+[latest_volume]))

### Chaikin Oscillator
The Chaikin Oscillator calculates a long and short period of the accumulation period, takes their moving average, and returns the difference. The period for the long period is 10, and 3 for the short period.
This should be plotted against the `accumulation_distribution_indicator` function, where the Chaikin line being above the accumulation distribution indicator signifies a bull market, and being under is a bear market.

In [ ]:
chaikin_oscillator = bulk_oscillators.chaikin_oscillator(data['High'], data['Low'], data['Close'], data['Volume'])

print(chaikin_oscillator)

chaikin_oscillator.append(single_oscillators.chaikin_oscillator(
    data['High'][-9:] + [latest_high],
    data['Low'][-9:] + [latest_low],
    data['Close'][-9:] + [latest_close],
    data['Volume'][-9:] + [latest_volume]))

# TODO: When charting chart against the accumulation distribution indicator

### Personalised Chaikin Oscillator
The personalised Chaikin Oscillator allows the caller to determine the long and short period, as well as which moving average model to use.

The single version doesn't require a long version as it assumes that the length of the submitted lists are the long period.

In [ ]:
personalised_co = bulk_oscillators.personalised_chaikin_oscillator(data['High'], data['Low'], data['Close'], data['Volume'], 5, 20, 'ema')

print(personalised_co)

personalised_co.append(single_oscillators.personalised_chaikin_oscillator(
    data['High'][-19:] + [latest_high],
    data['Low'][-19:] + [latest_low],
    data['Close'][-19:] + [latest_close],
    data['Volume'][-19:] + [latest_volume], 5, 'ema'))

### Williams %R
Williams %R is calculated by looking at the difference between the high, close, and low.

Prices between 0 and -20 are considered overbought, prices between -80 and -100 are considered oversold

In [ ]:
weekly_williams_percent_r = bulk_oscillators.williams_percent_r(data['High'], data['Low'], data['Close'], 5)
monthly_williams_percent_r = bulk_oscillators.williams_percent_r(data['High'], data['Low'], data['Close'], 20)
quarterly_williams_percent_r = bulk_oscillators.williams_percent_r(data['High'], data['Low'], data['Close'], 60)

print(weekly_williams_percent_r)
# TODO: Either max here or do it in single
weekly_williams_percent_r.append(single_oscillators.williams_percent_r(latest_high, latest_low, latest_close))

## Strength Indicators
Strength indicators show how strong the trend (price movement), or the momentum is.

In [ ]:
from PyTechnicalIndicators.Bulk import strength_indicators as bulk_strength_indicators
from PyTechnicalIndicators.Single import strength_indicators as single_strength_indicators

### Relative Strength Index
The Relative Strength Index (RSI) shows how strong the price momentum of the asset is.

It looks at the 14 previous periods of the price.

In [ ]:
rsi = bulk_strength_indicators.relative_strength_index(data['Typical Price'])

print(rsi)

rsi.append(single_strength_indicators.relative_strength_index(data['Typical Price'][-13:] + [latest_typical_price]))

### Personalised RSI
The personalised RSI allows the caller to determine the period rather than having it look at the 14 previous values. It also allows the caller to determine the moving average model being used.

For the single version the period is determined from the length of the submitted list.

In [ ]:
weekly_rsi = bulk_strength_indicators.personalised_rsi(data['Typical Price'], 5, 'ema')
monthly_rsi = bulk_strength_indicators.personalised_rsi(data['Typical Price'], 20, 'ema')
quarterly_rsi = bulk_strength_indicators.personalised_rsi(data['Typical Price'], 60, 'ema')

print(weekly_rsi)

weekly_rsi.append(single_strength_indicators.personalised_rsi(data['Typical Price'][-4:] + [latest_typical_price], 'ema'))
monthly_rsi.append(single_strength_indicators.personalised_rsi(data['Typical Price'][-19:] + [latest_typical_price], 'ema'))
quarterly_rsi.append(single_strength_indicators.personalised_rsi(data['Typical Price'][-59:] + [latest_typical_price], 'ema'))

### Accumulation Distribution Indicator
The accumulation distribution indicator (ADI) shows the link between the price and the volume. By doing so it tries to determine the strength of the trend. A rising line in combination with rising confirms an upward trend.

The ADI uses the previously calculated ADI to get the new value, this is passable when calling the single function.

In [ ]:
adi = bulk_strength_indicators.accumulation_distribution_indicator(data['High'], data['Low'], data['Close'], data['Volume'])

print(adi)

adi.append(single_strength_indicators.accumulation_distribution_indicator(
    latest_high,
    latest_low,
    latest_close,
    latest_volume,
    adi[-1]
))

### Directional Indicator, Directional Index, Average Directional Index, Average Directional Index Rating
The next three indicators are grouped into one as they all depend on one another and essentially create a combined indicator.
These indicators try to give the caller an idea of the direction of the price as well as the strength of the trend
The directional indicator (DI) returns the positive and negative directional indicators, as well as the true range. This returns a list of tuples, so will need to be extracted before passing it to the DX.
The directional index (DX) takes the positive and negative DI as parameters.
The average directional index (ADX) takes the DX as a parameter as well as a period and a moving average model

In [ ]:
# TODO: Improve the description, go to Welles book. Get Welles default, add explanation about period DI
weekly_di = bulk_strength_indicators.directional_indicator(data['High'], data['Low'], data['Close'], 5)
weekly_positive_di = [di[0] for di in weekly_di]
weekly_negative_di = [di[1] for di in weekly_di]
weekly_dx = bulk_strength_indicators.directional_index(weekly_positive_di, weekly_negative_di)
weekly_adx = bulk_strength_indicators.average_directional_index(weekly_dx, 5, 'ema')
weekly_adxr = bulk_strength_indicators.average_directional_index_rating(weekly_adx, 5)

monthly_di = bulk_strength_indicators.directional_indicator(data['High'], data['Low'], data['Close'], 20)
monthly_positive_di = [di[0] for di in monthly_di]
monthly_negative_di = [di[1] for di in monthly_di]
monthly_dx = bulk_strength_indicators.directional_index(monthly_positive_di, monthly_negative_di)
monthly_adx = bulk_strength_indicators.average_directional_index(weekly_dx, 20, 'ema')
monthly_adxr = bulk_strength_indicators.average_directional_index_rating(weekly_adx, 20)

quarterly_di = bulk_strength_indicators.directional_indicator(data['High'], data['Low'], data['Close'], 60)
quarterly_positive_di = [di[0] for di in weekly_di]
quarterly_negative_di = [di[1] for di in weekly_di]
quarterly_dx = bulk_strength_indicators.directional_index(quarterly_positive_di, quarterly_negative_di)
quarterly_adx = bulk_strength_indicators.average_directional_index(weekly_dx, 60, 'ema')
quarterly_adxr = bulk_strength_indicators.average_directional_index_rating(weekly_adx, 60)

weekly_di.append(single_strength_indicators.period_directional_indicator_known_previous(
    latest_high,
    data['High'][-1],
    latest_low,
    data['Low'][-1],
    data['Close'][-1],
    weekly_di[2],
    weekly_di[0],
    weekly_di[1],
    5
))
weekly_dx.append(single_strength_indicators.directional_index(weekly_di[-1][0], weekly_di[-1][1]))
weekly_adx.append(single_strength_indicators.average_directional_index(weekly_dx[-1], 'ema'))
weekly_adxr.append(single_strength_indicators.average_directional_index_rating(weekly_adx[-1], weekly_adx[-5]))

## Momentum Indicators

In [ ]:
from PyTechnicalIndicators.Bulk import momentum_indicators as bulk_momentum_indicators
from PyTechnicalIndicators.Single import momentum_indicators as single_momentum_indicators

### Rate of Change
The rate of change (RoC) returns the percentage change between the price and the previous price.
This can be done any period.

In [ ]:
weekly_roc = bulk_momentum_indicators.rate_of_change(data['Typical Price'], 5)
monthly_roc = bulk_momentum_indicators.rate_of_change(data['Typical Price'], 20)
quarterly_roc = bulk_momentum_indicators.rate_of_change(data['Typical Price'], 60)

print(weekly_roc)

weekly_roc.append(single_momentum_indicators.rate_of_change(latest_typical_price, data['Typical Price'][-4:]))
monthly_roc.append(single_momentum_indicators.rate_of_change(latest_typical_price, data['Typical Price'][-19:]))
quarterly_roc.append(single_momentum_indicators.rate_of_change(latest_typical_price, data['Typical Price'][-59:]))

### On Balance Volume
The on balance volume (OBV) tries to show buying and selling pressure on an asset. 
It is calculated by looking at the change in volume in comparison to the close price.
The single version takes the previous OBV as an optional parameter.

In [ ]:
obv = bulk_momentum_indicators.on_balance_volume(data['Close'], data['Volume'])

print(obv)

obv.append(single_momentum_indicators.on_balance_volume(latest_close, data['Close'][-1], latest_volume, data['Volume'][-1]))

### Commodity Channel Index
The commodity channel Index (CCI) is a momentum indicator that takes the difference the latest price and the moving average and divides it by the absolute deviation.
The caller can determine the moving average model, absolute deviation model, and the period. These default the standards normally used to calculate the CCI.
The single version doesn't take a period, it is determined from the length of the price parameter.

In [ ]:
weekly_cci = bulk_momentum_indicators.personalised_commodity_channel_index(data['Typical Price'], 5, 'ema', 'median')
monthly_cci = bulk_momentum_indicators.personalised_commodity_channel_index(data['Typical Price'], 20, 'ema', 'median')
quarterly_cci = bulk_momentum_indicators.personalised_commodity_channel_index(data['Typical Price'], 60, 'ema', 'median')

print(weekly_cci)

weekly_cci.append(single_momentum_indicators.personalised_commodity_channel_index(data['Typical Price'][-4:] + [latest_typical_price], 'ema', 'median'))
monthly_cci.append(single_momentum_indicators.personalised_commodity_channel_index(data['Typical Price'][-19:] + [latest_typical_price], 'ema', 'median'))
quarterly_cci.append(single_momentum_indicators.personalised_commodity_channel_index(data['Typical Price'][-59:] + [latest_typical_price], 'ema', 'median'))

## Trend Indicators

In [ ]:
from PyTechnicalIndicators.Bulk import trend as bulk_trend_indicators
from PyTechnicalIndicators.Single import trend as single_trend_indicators

### Aroon Oscillator
The Aroon oscillator uses the aroon up and aroon down to determine the strength of the trend.
It oscilates between -100 and 100, anything above 0 could mean the beginning of an uptrend, or a downtrend if it goes under 0. Anything past -50 or 50 can signal big price movements.
By default it looks at 25 periods.

In [ ]:
# TODO: rework it

### Parabolic Stop and Reverse
The parabolic SAR attempts to give the caller a direction of the trend

In [ ]:
# TODO: PSAR needs to be reworked so that the bulk function returns values useable by the single function

## Candle Indicators
Candle indicators are used on candle charts to attempt to give buy and sell signals.

In [ ]:
from PyTechnicalIndicators.Bulk import candle_indicators as bulk_candle_indicators
from PyTechnicalIndicators.Single import candle_indicators as single_candle_indicators

### Bollinger Bands
Bollinger bands are very similar to moving average envelope bands, the only difference is that it adds or substracts the standard deviation to the moving average
A buy signal is when the price breaks the lower band, and sell signal is when the upper band is broken.
The default parameters are to use the moving average as the ma model and to use 2 standard deviations for the addition/substraction, and looks at 20 periods. But these can be changed by the caller.
The single function doesn't take a period but determines it from the length of the price list

In [ ]:
weekly_bband = bulk_candle_indicators.personalised_bollinger_bands(data['Typical Price'], 5, 'ema', 3)
monthly_bband = bulk_candle_indicators.personalised_bollinger_bands(data['Typical Price'], 20, 'ema', 3)
quarterly_bband = bulk_candle_indicators.personalised_bollinger_bands(data['Typical Price'], 60, 'ema', 3)

print(weekly_bband)

weekly_bband.append(single_candle_indicators.personalised_bollinger_bands(
    data['Typical Price'][-4:] + [latest_typical_price],
    'ema',
    3
))
monthly_bband.append(single_candle_indicators.personalised_bollinger_bands(
    data['Typical Price'][-19:] + [latest_typical_price],
    'ema',
    3
))
quarterly_bband.append(single_candle_indicators.personalised_bollinger_bands(
    data['Typical Price'][-59:] + [latest_typical_price],
    'ema',
    3
))

### Ichimoku Cloud
The Ichimoku cloud tries to show support/resistance levels as well as general trend direction.

The idea is that when span A is over span B there is potential for an uptrend, this is reinforced is the price is above the cloud.

The default periods for this are based on a 6 day work week, the conversion period defaults to 9, the base period defaults to 26, and the span b period defaults to 52. These can be changed by the caller.


In [ ]:
# TODO: Return the base line if it is needed to determine price direction
ichimoku_cloud = bulk_candle_indicators.personalised_ichimoku_cloud(
    data['High'],
    data['Low'],
    5,
    20,
    60
)

print(ichimoku_cloud)

ichimoku_cloud.append(
    data['High'][-59:] + [latest_high],
    data['Low'][-59:] + [latest_close],
    5,
    20,
    60
)

## Volatility

In [ ]:
from PyTechnicalIndicators.Bulk import volatility as bulk_volatility_indicators
from PyTechnicalIndicators.Single import volatility as single_volatility_indicators

### Average True Range
The average true range (ATR) gives the caller an idea of the range or spread of the prices over a given period. The higher the range, the more volatile the price.

In the single function the length of the passed prices is used to determine the period. There are also two functions for the average true range, one where the previous true range is known `average_true_range` and one where the initial is not known `average_true_range_initial` which should only be used to initialize the ATR list.

In [ ]:
# TODO: the close price needs to be shifted one as it should be t-1 close
weekly_atr = bulk_volatility_indicators.average_true_range(
    data['High'],
    data['Low'],
    data['Close'],
    5
)
monthly_atr = bulk_volatility_indicators.average_true_range(
    data['High'],
    data['Low'],
    data['Close'],
    5
)
quarterly_atr = bulk_volatility_indicators.average_true_range(
    data['High'],
    data['Low'],
    data['Close'],
    5
)

print(weekly_atr)

weekly_atr.append(single_volatility_indicators.average_true_range(latest_high, latest_low, data['Close'][-1], weekly_atr[-1], 5))
monthly_atr.append(single_volatility_indicators.average_true_range(latest_high, latest_low, data['Close'][-1], weekly_atr[-1], 20))
quarterly_atr.append(single_volatility_indicators.average_true_range(latest_high, latest_low, data['Close'][-1], weekly_atr[-1], 60))

### Ulcer Index
The Ulcer index (UI) calculates volatility by looking at the difference between the period high and latest price. Peaks that are higher than the average denote high volatility.

The default period is 14 days but can be changed by the caller.

The single function determines the period from the list of prices.

In [ ]:
weekly_ui = bulk_volatility_indicators.ulcer_index(data['Close'], 5)
monthly_ui = bulk_volatility_indicators.ulcer_index(data['Close'], 20)
quarterly_ui = bulk_volatility_indicators.ulcer_index(data['Close'], 60)

print(weekly_ui)

weekly_ui.append(data['Close'][-4:] + [latest_close])
monthly_ui.append(data['Close'][-19:] + [latest_close])
quarterly_ui.append(data['Close'][-59:] + [latest_close])

### Welles Volatility Index
Welles developed his volatility index (VI) to observe volatility of assets.

The single function takes the latest value of the volatility index as a parameter if it is available.


In [ ]:
# TODO: Go to book to get better description, the internet has very little
weekly_vi = bulk_volatility_indicators.volatility_index(
    data['High'],
    data['Low'],
    data['Close'],
    5
)
monthly_vi = bulk_volatility_indicators.volatility_index(
    data['High'],
    data['Low'],
    data['Close'],
    20
)
quarterly_vi = bulk_volatility_indicators.volatility_index(
    data['High'],
    data['Low'],
    data['Close'],
    60
)

print(weekly_vi)

weekly_vi.append(single_volatility_indicators.volatility_index(
    latest_high,
    latest_low,
    latest_close,
    5,
    weekly_vi[-1]
))
monthly_vi.append(single_volatility_indicators.volatility_index(
    latest_high,
    latest_low,
    latest_close,
    5,
    monthly_vi[-1]
))
quarterly_vi.append(single_volatility_indicators.volatility_index(
    latest_high,
    latest_low,
    latest_close,
    5,
    quarterly_vi[-1]
))

### Welles Volatility System
Same go to book for more info

In [ ]:
weekly_vs = bulk_volatility_indicators.volatility_system(
    data['High'],
    data['Low'],
    data['Close'],
    5,
    2
)
monthly_vs = bulk_volatility_indicators.volatility_system(
    data['High'],
    data['Low'],
    data['Close'],
    20,
    2
)
quarterly_vs = bulk_volatility_indicators.volatility_system(
    data['High'],
    data['Low'],
    data['Close'],
    60,
    2
)

print(weekly_vs)

weekly_vs.append(single_volatility_indicators.volatility_system(
    data['High'][-4:] + [latest_high],
    data['Low'][-4:] + [latest_low],
    data['Close'][-4:] + [latest_close],
    5,
    2,
    weekly_vs[-1]
))
monthly_vs.append(single_volatility_indicators.volatility_system(
    data['High'][-19:] + [latest_high],
    data['Low'][-19:] + [latest_low],
    data['Close'][-19:] + [latest_close],
    20,
    2,
    monthly_vs[-1]
))
quarterly_vs.append(single_volatility_indicators.volatility_system(
    data['High'][-59:] + [latest_high],
    data['Low'][-59:] + [latest_low],
    data['Close'][-59:] + [latest_close],
    59,
    2,
    quarterly_vs[-1]
))

### Correlation

In [ ]:
from PyTechnicalIndicators.Bulk import correlation as bulk_correlation_indicators
from PyTechnicalIndicators.Single import correlation as single_correlation_indicators

### Correlate asset prices
A simple function that allows the caller to correlate two asset prices for a given period.

A value of 1 means that assets are perfectly correlated, the prices move in the exact same way. 
A value of -1 means that they are perfectly uncorrelated, the prices move in opposite directions.

For the single function the period is determined from the length of prices

In [ ]:
# TODO: get another asset to do the correlation here
correlation = bulk_correlation_indicators.correlate_asset_prices(data['Typical Price'], data['Typical Price'], 5)

print(correlation)

correlation.append(single_correlation_indicators.correlate_asset_prices(
    data['Typical Price'][-4:] + [latest_typical_price], 
    data['Typical Price'][-4:] + [latest_typical_price]
))

### Other indicators


# Chart Patterns
The chart patterns are a collection of indicators made to be displayed on the OHLC charts to shows trends.

## Peaks

In [ ]:
from PyTechnicalIndicators.Chart_Patterns import peaks

### Get peaks

# THINGS TO FIX
* use the python median
* rename stddev to standard deviation
* add moving mode, median, MCGin... 
* as well as their envelopes
* check how correct the MA envelope is, should it be the bracket of the current period or the previous period? as in it should be upper and lower of the t-1 period to show how t aligns. Looks like the code is doing that but it needs to be confirmed
* chaikin oscillator defaul ma model may be wrong
* in accumulation distribution indicator instantiate the list with the first value rather than having an if statement in the loop
* Close prices need to be shifted because they are currently the close prices at t but should be for t-1, create new previous close column. Make sure that all indicators that use close handle it as expected
* See where the directional indicator function is being used or if it should just be replaced with the period DI
* rename trend.py to trend_indicators.py?
* on balance volume and commodity channel index need docstrings
* replace standard versions with the personalised just having the defaults so RSI with pers RSI just having params as filled in case callers don't want to change it
* Some TI are missing period defaults
* for aroon have the function figure out the time since last high and low based on the provided period.
* bband should call personalised
* rename params to just be price, caller can put whatever price they want in there
* icloud may need the baseline to also be returned?
* rename volatility to volatility indicators?
* volatility system is missing the doc string for average true range constant
* williams %R needs to either do the max in single function or doc string needs to tell people to do the max for the period before passing it on
* rename correlation to correlation indicators?
* bulk other > value added personalised needs a period (and to be renamed)